<a href="https://colab.research.google.com/github/pravsels/DistilLM/blob/main/manim/HF_Training_Script_mp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install datasets transformers accelerate peft bitsandbytes

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 10.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 35.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.9/190.9 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 16.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00


In [2]:
!apt install git-lfs

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 35 not upgraded.


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import transformers
from datasets import load_dataset
import math
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments

In [3]:
hf_dataset = load_dataset('pravsels/manim_pile')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


In [4]:
hf_dataset['train'], hf_dataset['eval'] = hf_dataset['train'].train_test_split(.1).values()

hf_dataset

DatasetDict({
    train: Dataset({
        features: ['metadata', 'text'],
        num_rows: 6975
    })
    eval: Dataset({
        features: ['metadata', 'text'],
        num_rows: 776
    })
})

In [5]:
hf_dataset['train'][11]

{'metadata': {'dataset_name': 'mediciresearch/manimation', 'filename': 'N/A'},
 'text': '<s>[INST] Generate accurate and correct ManimCE Python code for the animation requested by the user. Here is the user\'s request: Can you create an educational animation using ManimCE that illustrates the concept of simple harmonic motion? I would like to start by showing the differential equation \\( m\\frac{d^2 x}{dt^2} = -kx \\) written out on the screen. Then, can you add a brief explanation of what each term represents (m for mass, k for spring constant, and x for position) just above the equation? After that, please replace the differential equation with the form of its solution \\( x(t) = A \\cos(\\omega t - \\phi) \\) and provide a similar explanation for the new terms (A for amplitude, omega for angular frequency, and phi for phase shift) above it. Lastly, could you animate a circle moving back and forth like an oscillator, with an arrow showing the force direction, and then go back to dis

In [6]:
hf_dataset['eval'][11]

{'metadata': {'dataset_name': 'pravsels/manim_3b1b_issues', 'filename': 'N/A'},
 'text': 'Add type hints according to PEP 484 and PEP 604 : Add type hints according to [PEP 484](https://www.python.org/dev/peps/pep-0484/), [PEP 526](https://www.python.org/dev/peps/pep-0526/) and [PEP 604](https://www.python.org/dev/peps/pep-0604/)\r\n\r\n## Motivation\r\n\r\nThe motivation for this is to allow the IDE (language server) to better analyze the code and give type hints like:\r\n<img width="715" alt="image" src="https://user-images.githubusercontent.com/44120331/154065308-c147997d-31ac-4eea-b231-b15633b8a68f.png">\r\nAnd this also provides convenience for writing API documentation in the next step\r\n\r\n## Changes\r\n\r\nI manually wrote type annotations for almost all class methods and functions following these PEP standards. \r\nHowever, due to the dynamic addition of attributes using `CONFIG` and `digest_config`, the types of some attribute values \u200b\u200bcannot be annotated and ther

In [7]:
base_model = 'deepseek-ai/deepseek-coder-6.7b-instruct'
# max_context_size= 16384
# setting context length of out inputs
# block_size = tokenizer.model_max_length

block_size = 1024     # context len of input
# using the same tokenizer that was used for training the model
tokenizer = AutoTokenizer.from_pretrained(base_model,
                                          use_fast=True)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [8]:
def tokenize_func(examples):
  return tokenizer(examples['text'],
                   max_length=block_size,
                   truncation=True)

In [9]:
tokenized_dataset = hf_dataset.map(tokenize_func,
                                   batched=True,
                                   num_proc=8,
                                   remove_columns=['text', 'metadata'])

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

In [10]:
print(tokenized_dataset)
print(tokenized_dataset['train'][0])
print(tokenized_dataset['eval'][0])

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 6975
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 776
    })
})
{'input_ids': [32013, 32, 1752, 380, 905, 279, 7889, 13, 4016, 1191, 680, 2220, 5283, 62, 5066, 3442, 63, 280, 2220, 18074, 13, 4016, 63, 254, 7506, 2220, 1314, 415, 63, 317, 6350, 9685, 13, 997, 4059, 276, 330, 5452, 13, 7567, 5765, 0, 3119, 340, 1120, 276, 13109, 245, 10214, 327, 437, 30, 7589, 13, 303, 740, 1098, 274, 10214, 331, 437, 13], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}
{'input_ids': [32013, 24127, 254, 664, 308, 1959, 10601, 2194, 4368, 523, 265, 8962, 1160, 13, 9681, 13, 3221, 18479, 4926, 3305, 1191, 303, 604, 331, 4260, 13, 416, 37, 76, 18399, 285, 16596, 55, 8519, 55, 417, 2561, 3972, 331, 5

In [11]:
def group_text_func(examples):
  # for every key ('train', 'test' etc.), concatenate the texts

  concat_examples = {key: sum(examples[key], []) for key in examples.keys()}
  total_len = len(concat_examples[list(examples.keys())[0]])
  total_len = (total_len // block_size) * block_size

  # split by chunks of block_size
  result = {
      key: [text[i : i + block_size] for i in range(0, total_len, block_size)] for key, text in concat_examples.items()
  }
  result['labels'] = result['input_ids'].copy()

  return result

In [12]:
lm_dataset = tokenized_dataset.map(group_text_func,
                                   batched=True,
                                   batch_size=128,
                                   num_proc=8)
lm_dataset

Map (num_proc=8):   0%|          | 0/6975 [00:00<?, ? examples/s]

Map (num_proc=8):   0%|          | 0/776 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 3686
    })
    eval: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 394
    })
})

In [13]:
print('input ids : ', lm_dataset['train'][0]['input_ids'][:5])
print('attention mask : ', lm_dataset['train'][0]['attention_mask'][:5])
print('label ids : ', lm_dataset['train'][0]['labels'][:5])

input ids :  [32013, 32, 1752, 380, 905]
attention mask :  [1, 1, 1, 1, 1]
label ids :  [32013, 32, 1752, 380, 905]


In [14]:
from peft import LoraConfig

peft_config = LoraConfig(
    lora_alpha=16,
    lora_dropout=0.1,
    target_modules=["q_proj", "k_proj"],
    r=64,
    bias="none",
    task_type="CAUSAL_LM",
)

In [15]:
model = AutoModelForCausalLM.from_pretrained(
                                              base_model,
                                              device_map="auto",
                                              load_in_8bit=True
                                             )

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [16]:
model.add_adapter(peft_config,
                  adapter_name="manim_adapter")

In [17]:
import numpy as np
def count_trainable_parameters(model):
    model_parameters = filter(lambda p: p.requires_grad, model.parameters())
    params = sum([np.prod(p.size()) for p in model_parameters])
    total_params = sum([np.prod(p.size()) for p in model.parameters()])
    return params/total_params

count_trainable_parameters(model) * 100

0.49533656825843125

In [18]:
training_args = TrainingArguments(
  f"{base_model}_finetuned_manim_pile_context_{block_size}",
  evaluation_strategy = "epoch",
  # num_train_epochs=10,
  learning_rate=2e-5,
  per_device_train_batch_size=1,
  per_device_eval_batch_size=1,
  gradient_accumulation_steps=8,
  weight_decay=0.01,
  push_to_hub=True
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_dataset['train'],
    eval_dataset=lm_dataset['eval']
)

In [ ]:
trainer.train()

Epoch,Training Loss,Validation Loss


In [ ]:
eval_results = trainer.evaluate()
print(f"Perplexity score : {math.exp(eval_results['eval_loss']):.2f}")

In [ ]:
model.save_pretrained('./')

In [ ]:
trainer.push_to_hub()

In [ ]:
model_name = 'pravsels/deepseek-coder-6.7b-instruct-finetuned-manimation'
tokenizer = AutoTokenizer.from_pretrained(base_model, use_fast=True)

In [ ]:
model = AutoModelForCausalLM.from_pretrained(model_name,
                                             device_map="auto",
                                             load_in_4bit=True)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [ ]:
gen_context_len = 512

input_text = "write manim code to transition a blue circle to a red square. please write code only, no explanation."
inputs = tokenizer(input_text, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=gen_context_len)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

Setting `pad_token_id` to `eos_token_id`:32021 for open-end generation.
/usr/local/lib/python3.10/dist-packages/bitsandbytes/nn/modules.py:226: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(f'Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.')


write manim code to transition a blue circle to a red square. please write code only, no explanation.


```python
from manim import *

class SquareToCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(BLUE, opacity=0.5)  # set the color and transparency
        self.play(Create(circle))  # show the circle on screen

        square = Square()  # create a square
        square.rotate(PI / 4)  # rotate a certain amount
        square.set_fill(RED, opacity=0.5)  # set the color and transparency
        self.play(Create(square))  # show the square on screen

        self.play(TransformAround(circle, square))  # interpolate circle to square
        self.wait()
```

This code creates a blue circle and a red square, then transitions the blue circle to the red square.

```python
from manim import *

class SquareToCircle(Scene):
    def construct(self):
        circle = Circle()  # create a circle
        circle.set_fill(BLUE, opacity=0.5)